# Three-Body Problem

## Introduction
This notebook implements a numeric simulation of the [three-body problem](http://en.wikipedia.org/wiki/Three-body_problem). This is 
done using the [Julia Language](http://julialang.org) and the [Sundials](https://github.com/JuliaLang/Sundials.jl) library. 

The problem corresponds to determine the trajectories of three bodies in space under the influence of their mutual gravity forces (e.g., the motion of the Sun, Moon and Earth, taken together). Isaac Newton was the first person to try to solve this problem. In 1887, Bruns and Poincaré showed that there is no general analytical closed solution for this problem. In fact, the motion of the three bodies is generally non-repeating! That's pretty amazing.

![bodies](http://37.media.tumblr.com/a7089fb9c639a8a13dc750cf4d79ffaf/tumblr_n4iuk3m70A1tzs5dao1_1280.gif)


---

## Universal Law of Gravitation

Newton taught us that two bodies with masses $m_0$ and $m_1$ attract themselves with a force proportional to their masses and inversionally proportional to the square of their distance $d$. This is called the [Universal Law of Gravitation](http://en.wikipedia.org/wiki/Newton's_law_of_universal_gravitation). Mathematically this is expressed by: 

$$ F = G \frac{m_0 \, m_1}{d^2} $$

where $G$ is the gravitational constant. But, if we want to consider the movement of three bodies in space, we'll need to formulate this in terms of vector calculus. Let's suppose that mass $m_0$ is at position $\vec{r_0}$ and $m_1$ at position $\vec{r_1}$. This is shown in the next diagram (which also shows a mass $m_2$ and its position -- for now, assume it doesn't exist):

![System](https://thibaudetienne.files.wordpress.com/2015/07/tbp.png)

In this case, the law of gravitation can be formulated as:

$$ \vec{F_{01}} = - G \frac{m_0 m_1}{{\vert \vec{r_0} - \vec{r_1} \vert}^2} \hat{r_{01}} $$

where $\vec{F_{01}}$ is the force exerted on $m_1$ due to $m_0$ and $\hat{r_{01}}$ is the unit vector from $m_0$ to $m_1$:

$$ \hat{r_{01}} = \frac{\vec{r_1} - \vec{r_0}}{\vert \vec{r_1} - \vec{r_0} \vert} $$

By simple manipulation we can write:

$$ \vec{F_{01}} = - G \, m_0 \, m_1 \frac{\vec{r_1} - \vec{r_0}}{{\vert \vec{r_1} - \vec{r_0} \vert}^3}  $$

Finally, there's an important thing that we also know: force equals mass times accelaration (Newton's second law) and that accelaration is the second derivative of position:

$$ \vec{F} = m \frac{d^2\vec{r}}{dt^2} $$

Thus we can write:

$$ m_1 \frac{d^2\vec{r_1}}{dt^2} = - G \, m_0 \, m_1 \frac{\vec{r_1} - \vec{r_0}}{{\vert \vec{r_1} - \vec{r_0} \vert}^3} $$

If we only had two bodies this would be great. But, with three bodies, we need to consider all the interactions and forces. Since we are calculating the accelaration of $m_1$ we just need to add the force corresponding to $\vec{F_{21}}$:

$$ m_1 \frac{d^2\vec{r_1}}{dt^2} = - G \, m_0 \, m_1 \frac{\vec{r_1} - \vec{r_0}}{{\vert \vec{r_1} - \vec{r_0} \vert}^3} \
                                   - G \, m_2 \, m_1 \frac{\vec{r_1} - \vec{r_2}}{{\vert \vec{r_1} - \vec{r_2} \vert}^3} $$

---
# Equations for Three Bodies

From the last equation above and by symmetry we get the follwing three equations:

$$ \frac{d^2\vec{r_0}}{dt^2} = G \, m_1 \frac{\vec{r_1} - \vec{r_0}}{{\vert \vec{r_1} - \vec{r_0} \vert}^3} + \
                               G \, m_2 \frac{\vec{r_2} - \vec{r_0}}{{\vert \vec{r_2} - \vec{r_0} \vert}^3} $$

$$ \frac{d^2\vec{r_1}}{dt^2} = G \, m_0 \frac{\vec{r_0} - \vec{r_1}}{{\vert \vec{r_0} - \vec{r_1} \vert}^3} + \
                               G \, m_2 \frac{\vec{r_2} - \vec{r_1}}{{\vert \vec{r_2} - \vec{r_1} \vert}^3} $$

$$ \frac{d^2\vec{r_2}}{dt^2} = G \, m_0 \frac{\vec{r_0} - \vec{r_2}}{{\vert \vec{r_0} - \vec{r_2} \vert}^3} + \
                               G \, m_1 \frac{\vec{r_1} - \vec{r_2}}{{\vert \vec{r_1} - \vec{r_2} \vert}^3} $$
                                   
These can be integrated numerically. Let's do so.

---

Let's start by importing a ploting library and a differential equation solver.

In [ ]:
Pkg.add("Sundials")

In [ ]:
using PyPlot
using Sundials


Now we need to define a function $\dot{g} = f(t, g)$ where $g$ is a function of time $t$ and $\dot{g}$ represents its derivative. Using the Sundials library this is done by defining a function ```f(t, g, gdot)```. The function should update the return parameter ```gdot``` with the derivative of ```g(t)``` at time ```t```. Note that this design is not particularly great -- it would be much better if we could just return ```gdot```. (Note that typically $\dot{g} = f(t, g)$ is written as $\dot{y} = f(t, y)$. We are not doing so because we'll be using $x$ and $y$ to denote coordinates.)

Looking at the equations above it's easy to see that both ```y``` and ```ydot``` are going to be 12-entry vectors. We are going to work in two dimentions: X and Y. Thus, each vector $\vec{r_k(t)}$ is going to correspond to two coordinates: $\vec{r_k(t)} = (x_k(t), y_k(t))$. Since we have 3 of these position vectors this corresponds to 6 entries. Since the differencial equations above are of second order, we will need the derivatives of each one of $\vec{r_k(t)}$, which corresponds to the bodies velocities $\vec{v_k(t)}$. Since we have 3 bodies, each with an X and Y velocity, we get 6 other entries.

So, let's write this function using the 3 differential equations above. For simplicity, we'll assume that $G=1$.

In [ ]:
function f(t, g, gdot)
    # Extract the position and velocity vectors from the g array
    r0, v0 = g[1:2], g[3:4]
    r1, v1 = g[5:6], g[7:8]
    r2, v2 = g[9:10], g[11:12]
    
    # The derivatives of the position are simply the velocities
    dr0 = v0
    dr1 = v1
    dr2 = v2
    
    # Now calculate the the derivatives of the velocities, which are the accelarations
    # Start by calculating the distance vectors between the bodies (assumes m0, m1 and m2 are global variables)
    # Slightly rewriten the expressions dv0, dv1 and dv2 comprared to the normal equations so we can reuse d0, d1 and d2
    d0  = (r2 - r1) / ( norm(r2 - r1)^3.0 )
    d1  = (r0 - r2) / ( norm(r0 - r2)^3.0 )
    d2  = (r1 - r0) / ( norm(r1 - r0)^3.0 )    
    
    dv0 = m1*d2 - m2*d1
    dv1 = m2*d0 - m0*d2
    dv2 = m0*d1 - m1*d0
    
    # Reconstruct the derivative vector
    gdot[:] = [dr0; dv0; dr1; dv1; dr2; dv2]
end;

For running a simulation we just need to setup the mass of each body, its initial position and velocity. As a sanity check we are also calculating the center of gravity at the end, for each time step, to make sure it's not "drifting" (which would violate the laws of physics).

Here we use the unit of lenght to be [L]=1 AU (Sun-Earth distance) and the unit of mass to be the mass of the Sun [M]=$1M_{\odot}$. Under these choices and for $G=1$ the unit of time [T]=58 days.



# Simulate the Sun-Earth-Jupiter system

* Set the masses m0,m1,m2 such that m0=Sun Mass, m1=Earth Mass and m2=Jupiter Mass
  (google it!).
* Set the semi-major axis of Earth and Jupiter in our solar system.
  There correspond to the parameters a_earth,a_jupiter below.
  Set the planets initially in circular orbits such that e_earth=e_jupiter=0.0.

## A very eccentric Jupiter!

* Keeping all the other parameters constant set the Jupiter to be in very eccentric otbit  of eccentricity e_jupiter=0.9. What happened? 


## A very close Jupiter!

* Keeping all the other parameters constant set the Jupiter to be in a close circular otbit  of semi-major axis a_jupiter=1.2. Is the system stable?

## An Earth with two Suns!!

* Keeping all the other parameters constant set the Jupiter to as heavy as the Sun i.e., m2=1.0. What do you observe?


In [ ]:
# Masses of the bodies
m0 = 
m1 = 
m2 = 

e_earth=
e_jupiter=
a_earth=
a_jupiter=


# Initial positions and velocities of each body (x0, y0, vx0, vy0) 
factor_earth=sqrt((1.0+e_earth)/(1.0-e_earth))
factor_jupiter=sqrt((1.0+e_jupiter)/(1.0-e_jupiter))
gi0_pre = [ 0.0; 0.0; 0.0; 0.0]  
gi1_pre = [ 0.0; a_earth*(1.0-e_earth); factor_earth*sqrt(1.0/a_earth); 0.0]
gi2_pre =[ 0.0; a_jupiter*(1.0-e_jupiter); factor_jupiter*sqrt(1.0/a_jupiter); 0.0]


cxi = [(gi0_pre[1]*m0 + gi1_pre[1]*m1 + gi2_pre[1]*m2) / (m0 + m1 + m2)]
cyi = [(gi0_pre[2]*m0 + gi1_pre[2]*m1 + gi2_pre[2]*m2) / (m0 + m1 + m2)]
vcxi = [(gi0_pre[3]*m0 + gi1_pre[3]*m1 + gi2_pre[3]*m2) / (m0 + m1 + m2)]
vcyi = [(gi0_pre[4]*m0 + gi1_pre[4]*m1 + gi2_pre[4]*m2) / (m0 + m1 + m2)]


gi0 = [ 0.0-cxi; 0.0-cyi; 0.0-vcxi; 0.0-vcyi]  
gi1 = [ 0.0-cxi; a_earth*(1.0-e_earth)-cyi; factor_earth*sqrt(1.0/a_earth)-vcxi; 0.0-vcyi]
gi2 = [0.0-cxi; a_jupiter*(1.0-e_jupiter)-cyi; factor_jupiter*sqrt(1.0/a_jupiter)-vcxi; 0.0-vcyi]


# Simulation from time t=0 to t=tf
tf  = 10000.0
dt  = 10.
t   = collect(linspace(0.0, tf, round(Int,tf*dt)))
g0  = [gi0; gi1; gi2]
res = Sundials.cvode(f, g0, t, reltol=1e-14)

# Extract the position and velocity vectors from the solution
r0, v0, r1, v1, r2, v2 = res[:,1:2], res[:,3:4], res[:,5:6], res[:,7:8], res[:,9:10], res[:,11:12]

# Calculate the center of mass
cx = [(r0[i,1]*m0 + r1[i,1]*m1 + r2[i,1]*m2) / (m0 + m1 + m2) for i=1:length(t)]
cy = [(r0[i,2]*m0 + r1[i,2]*m1 + r2[i,2]*m2) / (m0 + m1 + m2) for i=1:length(t)]


# Write the results to file if we want to do something with them later on (e.g., create some videos)
# writecsv("results.csv", res);

Now we need a function that allows us to plot the trajectory over time. We'll call it ```plot_trajectory``` and will pass as parameter the start time and stop time to plot.

In [ ]:
function plot_trajectory(t1, t2)
    t1i = round(Int,(length(t)-1) * t1/tf) + 1
    t2i = round(Int,(length(t)-1) * t2/tf) + 1
    
    # Plot the initial and final positions
    # In these vectors, the first coordinate will be X and the second Y
    X = 1
    Y = 2
    
    figure(figsize=(6,6))
    plot(r0[t1i,X], r0[t1i,Y], "ro")
    plot(r0[t2i,X], r0[t2i,Y], "rs")
    plot(r1[t1i,X], r1[t1i,Y], "go")
    plot(r1[t2i,X], r1[t2i,Y], "gs")
    plot(r2[t1i,X], r2[t1i,Y], "bo")
    plot(r2[t2i,X], r2[t2i,Y], "bs")
    
    # Plot the trajectories
    plot(r0[t1i:t2i,X], r0[t1i:t2i,Y], "r-")
    plot(r1[t1i:t2i,X], r1[t1i:t2i,Y], "g-")
    plot(r2[t1i:t2i,X], r2[t1i:t2i,Y], "b-")
    
    # Plot cente of mass
    plot(cx[t1i:t2i], cy[t1i:t2i], "kx")
    
    # Setup the axis and titles
    xmin = minimum([r0[t1i:t2i,X]; r1[t1i:t2i,X]; r2[t1i:t2i,X]]) * 1.10
    xmax = maximum([r0[t1i:t2i,X]; r1[t1i:t2i,X]; r2[t1i:t2i,X]]) * 1.10
    ymin = minimum([r0[t1i:t2i,Y]; r1[t1i:t2i,Y]; r2[t1i:t2i,Y]]) * 1.10
    ymax = maximum([r0[t1i:t2i,Y]; r1[t1i:t2i,Y]; r2[t1i:t2i,Y]]) * 1.10
    
    axis([xmin, xmax, ymin, ymax])
    axis([-10., 10., -10., 10.])
    title(@sprintf "3-body simulation for t=[%.1f .. %.1f]" t1 t2)
end;

Let's plot the trajectory from 20 to 1000 time units.

In [ ]:
plot_trajectory(20, 1000);

The trajectory of each body is drawn in a different color. The start position is marked by a small circle and the end position by a square. The center of gravity is marked with a black X. As you can see the behavior is pretty insane. "Almost collisions" and unstable aperiodic trajectories.

And it goes on and on...

Let's plot the trajectory from 20 to 6000 time units.

In [ ]:
plot_trajectory(20, 6000);

Let's plot the trajectory from 0 to tf time units.

In [ ]:
plot_trajectory(0, tf);

That's pretty crazy! The earth was ejected by the Jupiter during their encounter!! We are now left only with the Sun and Jupiter!!